In [1]:
!pip install pip -qU
!pip install -r requirements.txt -q

In [2]:
import torch
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch import nn

import mlflow.pytorch
from mlflow.models import infer_signature


class IrisClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.2)
        x = self.fc3(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def prepare_data():
    iris = load_iris()
    data = iris.data
    labels = iris.target
    target_names = iris.target_names

    X_train, X_test, y_train, y_test = train_test_split(
        data, labels, test_size=0.2, random_state=42, shuffle=True, stratify=labels
    )

    X_train = torch.FloatTensor(X_train).to(device)
    X_test = torch.FloatTensor(X_test).to(device)
    y_train = torch.LongTensor(y_train).to(device)
    y_test = torch.LongTensor(y_test).to(device)

    return X_train, X_test, y_train, y_test, target_names


def train_model(model, epochs, X_train, y_train):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(epochs):
        out = model(X_train)
        loss = criterion(out, y_train).to(device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print("number of epoch", epoch, "loss", float(loss))

    return model


def test_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predict_out = model(X_test)
        _, predict_y = torch.max(predict_out, 1)

        print("\nprediction accuracy", float(accuracy_score(y_test.cpu(), predict_y.cpu())))
        return predict_out



In [3]:
import os
import json
import mlflow
import tempfile

MLFLOW_ROUTE = os.getenv("MLFLOW_ROUTE")
mlflow.set_tracking_uri(MLFLOW_ROUTE)
mlflow.set_experiment("iris")
mlflow.pytorch.autolog(registered_model_name="iris")


In [4]:
epochs = 100
model = IrisClassifier()
model = model.to(device)
X_train, X_test, y_train, y_test, target_names = prepare_data()
scripted_model = torch.jit.script(model)  # scripting the model
scripted_model = train_model(scripted_model, epochs, X_train, y_train)
# signature = test_model(scripted_model, X_test, y_test)
predict_out = test_model(scripted_model, X_test, y_test)
signature = infer_signature(X_test.numpy(), predict_out.detach().numpy())

with mlflow.start_run() as run:
    mlflow.pytorch.log_model(
        scripted_model, "model", signature=signature, backend="torchscript"
    )  # logging scripted model
    
    temp_model_dir = tempfile.TemporaryDirectory().name + "/pytorch-model"
    os.makedirs(temp_model_dir, exist_ok=True)
    
    scripted_model.save(temp_model_dir + "/model.pt")
    # print(scripted_model.graph)
    mlflow.log_artifact(temp_model_dir + "/model.pt", artifact_path="pytorch-model/1")
    
    # Generate ModelMesh Triton's config.pbtxt file
    triton_schema_text = f"""
    platform: "pytorch_libtorch"
    max_batch_size: 1
    input [
      {{
        name: "input__0"
        data_type: TYPE_FP32
        dims: {list(X_test.shape)}
      }}
    ]
    output [
      {{
        name: "output__0"
        data_type: TYPE_FP32
        dims: {list(predict_out.shape)}
      }}
    ]
    """
    # Save the Triton config.pbtxt
    schema_file_path_txt = temp_model_dir + "/config.pbtxt"  
    with open(schema_file_path_txt, "w") as f:
        f.write(triton_schema_text)
        
    mlflow.log_artifact(schema_file_path_txt, artifact_path="pytorch-model")


number of epoch 0 loss 1.0797507762908936
number of epoch 10 loss 0.8617256879806519
number of epoch 20 loss 0.6191173791885376
number of epoch 30 loss 0.45033156871795654
number of epoch 40 loss 0.318576842546463
number of epoch 50 loss 0.28423839807510376
number of epoch 60 loss 0.22784046828746796
number of epoch 70 loss 0.14465157687664032
number of epoch 80 loss 0.1809622347354889
number of epoch 90 loss 0.144547700881958

prediction accuracy 0.9666666666666667


2023/12/04 07:46:09 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/12/04 07:46:14 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
/opt/app-root/lib64/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
